In [19]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.compose import ColumnTransformer

In [2]:
ames = pd.read_csv("/Users/ben/Documents/GitHub/DSML/Data/AmesHousing.csv")
ames.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [3]:
X = ames.drop("SalePrice", axis = 1)
y = ames["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [4]:
# Size and Number of Rooms
lr = LinearRegression()

ct = ColumnTransformer(
    [("standardize", StandardScaler(),["Gr Liv Area", "TotRms AbvGrd"])],
    remainder = "drop"
)

my_pipeline = Pipeline([
    ("PreProcessing", ct),
    ("Regression", lr)
]).set_output(transform="pandas")

fitted_pipeline = my_pipeline.fit(X_train, y_train)

y_pred = fitted_pipeline.predict(X_test)

mse = mean_squared_error(y_true = y_test, y_pred = y_pred)
rmse = mse**(1/2)
print(f"Root Mean Squared Error: {rmse}")

scores = cross_val_score(fitted_pipeline, X, y, cv=5, scoring='neg_mean_squared_error')
avg_rmse = (-1*scores.mean())**(1/2)
print(f"Average Root Mean Squared Error: {avg_rmse}")

Root Mean Squared Error: 55007.2549234834
Average Root Mean Squared Error: 56001.24023779208


In [5]:
# Size, Number of Rooms, Building Type
ct = ColumnTransformer(
    [("Dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"]),
    ("standardize", StandardScaler(),["Gr Liv Area", "TotRms AbvGrd"])],
    remainder="drop"
)

my_pipeline = Pipeline([
    ("PreProcessing", ct),
    ("Regression", lr)
]).set_output(transform="pandas")

fitted_pipeline = my_pipeline.fit(X_train, y_train)

y_pred = fitted_pipeline.predict(X_test)

mse = mean_squared_error(y_true = y_test, y_pred = y_pred)
rmse = mse**(1/2)
print(f"Root Mean Squared Error: {rmse}")

scores = cross_val_score(fitted_pipeline, X, y, cv=5, scoring='neg_mean_squared_error')
avg_rmse = (-1*scores.mean())**(1/2)
print(f"Average Root Mean Squared Error: {avg_rmse}")

Root Mean Squared Error: 53314.11233013071
Average Root Mean Squared Error: 54311.98836516125


In [6]:
# Size, Building Type, and their interaction
ct = ColumnTransformer(
    [("Dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"]),
    ("standardize", StandardScaler(),["Gr Liv Area"])],
    remainder="drop"
).set_output(transform="pandas")

ct_inter = ColumnTransformer(
    [("Interaction", PolynomialFeatures(interaction_only=True), ["standardize__Gr Liv Area", "Dummify__Bldg Type_1Fam", "Dummify__Bldg Type_2fmCon", "Dummify__Bldg Type_Duplex", "Dummify__Bldg Type_Twnhs", "Dummify__Bldg Type_TwnhsE"])],
    remainder="drop"
).set_output(transform="pandas")

X_train_dummified = ct.fit_transform(X_train)
X_train_inter = ct_inter.fit_transform(X_train_dummified)

my_pipeline = Pipeline([
    ("PreProcessing", ct),
    ("Interact", ct_inter),
    ("Regression", lr)
]).set_output(transform="pandas")

fitted_pipeline = my_pipeline.fit(X_train, y_train)

y_pred = fitted_pipeline.predict(X_test)

mse = mean_squared_error(y_true = y_test, y_pred = y_pred)
rmse = mse**(1/2)
print(f"Root Mean Squared Error: {rmse}")

scores = cross_val_score(fitted_pipeline, X, y, cv=5, scoring='neg_mean_squared_error')
avg_rmse = (-1*scores.mean())**(1/2)
print(f"Average Root Mean Squared Error: {avg_rmse}")

Root Mean Squared Error: 52819.53982400751
Average Root Mean Squared Error: 53583.84060019522


In [7]:
# 5 Degree Polynomial on Size and Number of Rooms, with building type
ct = ColumnTransformer(
    [("Dummify", OneHotEncoder(sparse_output=False), ["Bldg Type"]),
     ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])],
    remainder = "drop"
).set_output(transform="pandas")

poly = PolynomialFeatures(degree = (1, 5))

ct_poly = ColumnTransformer(
    [("5-Degree Polynomial", poly, ["standardize__Gr Liv Area", "standardize__TotRms AbvGrd"])],
    remainder = "drop"
).set_output(transform = "pandas")

X_train_dummified = ct.fit_transform(X_train)
X_train_poly = ct_poly.fit_transform(X_train_dummified)

my_pipeline = Pipeline([
    ("PreProcessing", ct),
    ("Poly", ct_poly),
    ("Regression", lr)
]).set_output(transform="pandas")

fitted_pipeline = my_pipeline.fit(X_train, y_train)

y_pred = fitted_pipeline.predict(X_test)

mse = mean_squared_error(y_true = y_test, y_pred = y_pred)
rmse = mse**(1/2)
print(f"Root Mean Squared Error: {rmse}")

scores = cross_val_score(fitted_pipeline, X, y, cv=5, scoring='neg_mean_squared_error')
avg_rmse = (-1*scores.mean())**(1/2)
print(f"Average Root Mean Squared Error: {avg_rmse}")

Root Mean Squared Error: 57851.78264596086
Average Root Mean Squared Error: 78352.3821374158


The model with the lowest Root Mean Squared Error is the model with size, building type, and their interaction.\
After running cross validations on the four models, the model with the lowest average Root Mean Squared Error is the model with size, building type, and their interaction which is the same as we found before by calculating just one Root Mean Squared Error. 

In [13]:
ct = ColumnTransformer(
    [("Dummify", OneHotEncoder(sparse_output=False), ["Bldg Type"]),
     ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])],
    remainder = "drop"
).set_output(transform="pandas")

poly = PolynomialFeatures()

ct_poly = ColumnTransformer(
    [("Poly_1", poly, ["standardize__Gr Liv Area"]),
    ("Poly_2", poly, ["standardize__TotRms AbvGrd"])],
    remainder = "drop"
)

ct_poly_1 = ColumnTransformer(
    [("Polynomial_1", poly, ["standardize__Gr Liv Area"])],
    remainder = "passthrough"
).set_output(transform = "pandas")

ct_poly_2 = ColumnTransformer(
    [("Polynomial_2", poly, ["remainder__standardize__TotRms AbvGrd"])],
    remainder = "drop"
).set_output(transform = "pandas")

X_train_dummified = ct.fit_transform(X_train)
X_train_poly = ct_poly.fit_transform(X_train_dummified)
# X_train_poly_1 = ct_poly_1.fit_transform(X_train_dummified)
# X_train_poly_2 = ct_poly_2.fit_transform(X_train_poly_1)

# my_pipeline = Pipeline([
#     ("PreProcessing", ct),
#     ("Poly1", ct_poly_1),
#     ("Poly2", ct_poly_2),
#     ("Regression", lr)
# ]).set_output(transform="pandas")

my_pipeline = Pipeline([
    ("PreProcessing", ct),
    ("Poly", ct_poly),
    ("Regression", lr)
]).set_output(transform="pandas")

degrees = {'Poly__Poly_1__degree': list(range(1, 11)), "Poly__Poly_2__degree": list(range(1, 11))}

gscv = GridSearchCV(my_pipeline, degrees, cv = 5, scoring='r2')

In [14]:
gscv_fitted = gscv.fit(X, y)

In [21]:
scores = gscv_fitted.cv_results_['mean_test_score']

reshaped_scores = scores.reshape(-1, 10)

df = pd.DataFrame(reshaped_scores)

df.index = np.arange(1, len(df) + 1)

df.columns = [i for i in range(1, 11)]
df

,1,2,3,4,5,6,7,8,9,10
1,0.504209,0.506386,0.509102,0.514402,0.508661,0.513551,0.109585,-1.243942,0.225695,-1.718506
2,0.510919,0.506613,0.506505,0.507400,0.491352,0.504176,0.360343,-0.301577,0.457286,-0.251378
3,0.529559,0.530418,0.527478,0.523482,0.510005,0.509287,0.452479,0.383339,-1.146477,-64.898834
4,0.522860,0.526006,0.525562,0.530995,0.520465,0.516726,0.452285,0.384775,-1.340203,-67.134251
5,0.476571,0.489660,0.496708,0.515624,0.492705,0.419309,0.177101,-0.001650,-1.708776,-69.114476
6,0.427372,0.461563,0.473685,0.525176,0.516866,0.247598,-1.113622,-3.440775,-5.531080,-69.024411
7,0.515959,0.506099,0.478719,0.282558,-0.031128,0.387929,-0.330182,-5.203017,-11.353276,-78.811957
8,-1.274206,-1.482915,-1.663327,-3.664042,-4.715253,-2.227257,-0.051164,-6.513658,-25.254123,-111.896176
9,-44.096407,-45.397061,-46.331579,-50.623998,-61.639079,-53.738819,-38.330567,-20.756730,-32.837642,-107.503956
10,-529.003280,-517.904336,-519.703542,-541.599417,-569.397962,-548.834325,-503.570740,-479.702403,-532.579373,-925.573737


Q1: The best model is the model with polynomial degree 4 on both house size and number of rooms.

Q2: The downsides I see to trying all of the possible modeling options is that it increases the number of calculations that need to be ran, which increases the time. One way to reduce this would be to set a cutoff limit for $R^2$ values where you would stop testing polynomials of that size or larger if the cutoff value were reached. 